# Analyse all house of commons speeches since 1970

## **Part 1: Get a list of MPs and their affiliations**
[Part 2: Download all speeches belonging to MPs in list](MP_speeches-Part2.ipynb)

[Part 3: Train bigram and trigram models and use them on all speeches](MP_speeches-Part3.ipynb)

[Part 4: Train an LDA topic model and process all speeches with it](MP_speeches-Part4.ipynb)

[Part 5: Analyse the results of the LDA model](MP_speeches-Part5.ipynb)

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pickle

#### Download a list of MPs from TheyWorkForYou

In [2]:
if True:
    # Get MPs from all recent elections
    elections = ["1970-06-18", "1974-02-28", "1974-10-10", "1979-05-03", "1983-06-09", "1987-06-11",
                 "1992-04-09", "1997-05-01", "2001-06-07", "2005-05-05", "2010-05-06", "2015-06-07", "2017-05-01", "2017-06-15"]
    mp_dataframes = [pd.read_csv("https://www.theyworkforyou.com/mps/?f=csv&date="+election) for election in elections]
    # Group MPs by ID to remove duplicate entries
    mps = pd.concat(mp_dataframes).drop_duplicates(subset="Person ID", keep="last").sort_values("Person ID")

#### Read a list of women MPs and try to match with list of all MPs

In [3]:
if True:
    # Get list of women mps (from women-in-parliament R project)
    women_mps = pd.read_csv("mps_over_time.csv")
    # Match MPs by name to identify women in the list
    mps["full_name"] = mps["First name"].fillna("") + " " + mps["Last name"].fillna("")
    mps["clean_name"] = mps["full_name"].str.lower().replace(regex=True, to_replace=r'\W',value=r'')
    mps["is_female"] = mps["clean_name"].map(lambda x: x in women_mps["clean_name"].tolist())

In [4]:
if True:
    # Join women MP dataframe with mps dataframe to get party of women MPs
    mps = mps.join(women_mps[women_mps.party.notnull()][["clean_name", "party"]]\
                   .drop_duplicates(keep="last")\
                   .set_index("clean_name"),
                   on="clean_name").rename(columns={"party":"party_women"})

In [5]:
if True:
    # Replace party abbreviations with full names
    party_abbr = {
        "Lab":"Labour",
        "LD":"Liberal Democrat",
        "Con":"Conservative",
        "SNP":"Scottish National Party",
        "PC":"Plaid Cymru",
        "UU":"UUP",
        "Alliance":"Alliance",
        "SF":"Sinn Féin",
        "DU":"DUP",
        "SDLP":"Social Democratic and Labour Party",
        "SDP":"SDP",
        "Green":"Green",
        "Ind. Unity":"Independent"
    }

    mps.party_women = mps.party_women.apply(party_abbr.get)

#### Get list of MPs from MySociety's github and match with MP list using Person ID

In [6]:
if True:
    # Get json list of all MPs catalogued in the mysociety database
    mps_mysociety = requests.get("https://github.com/mysociety/parlparse/blob/master/members/people.json?raw=true").json()

    # Turn into pandas dataframes
    party_id_crossmatch = pd.DataFrame(mps_mysociety["organizations"])[["id", "name"]]
    mp_party_crossmatch = pd.DataFrame(mps_mysociety["memberships"]).dropna(subset=["on_behalf_of_id"])[["person_id", "on_behalf_of_id"]].drop_duplicates()

    # Match person_id with party names
    mp_party = mp_party_crossmatch.join(party_id_crossmatch.set_index("id"), on="on_behalf_of_id").groupby("person_id").first()[["name"]]

    # Match mps in our list with mps in publicwhip list to get party
    mps = mps.assign(mysoc=mps["Person ID"]\
                .apply(lambda x: "uk.org.publicwhip/person/" + str(x)))\
                .join(mp_party, on="mysoc", how="left")\
                .drop("mysoc", axis=1)\
                .rename(columns={"name":"party_mysoc"})

#### Get a list of all MPs from wikidata, then filter to MPs who are still alive after 1970

In [7]:
if True:
    import requests
    import pandas as pd
    # Get all mps that exist in wikidata.org
    wikidata_query = '''SELECT ?mp ?mpLabel ?dob ?dod ?party ?partyLabel ?hansard ?genderLabel WHERE {
      ?mp ?x1 wd:Q16707842; # Q16707842 = member of UK parliament
      wdt:P102 ?party. # P102 = belonging to a political party
      OPTIONAL {?mp wdt:P569 ?dob} . #P569 = date of birth
      OPTIONAL {?mp wdt:P570 ?dod} . #P570 = date of death
      OPTIONAL {?mp wdt:P2015 ?hansard} . #P2015 = hansard id
      OPTIONAL {?mp wdt:P21 ?gender} . #21 = gender

      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }'''

    wikidata_data = requests.get('https://query.wikidata.org/bigdata/namespace/wdq/sparql',
                                 params={'query': wikidata_query, 'format': 'json'}).json()

    def extract_optional(row, key):
        try:
            return row[key]["value"]
        except KeyError:
            return None

    # Convert the results to a pandas dataframe
    wikidata_mps = pd.DataFrame([ {"mp":mp["mpLabel"]["value"], "mp_id":mp["mp"]["value"],
                                   "gender":extract_optional(mp, "genderLabel"),
                                   "dob":extract_optional(mp, "dob"), "dod":extract_optional(mp, "dod"),
                                   "party": mp["partyLabel"]["value"], "party_id": mp["party"]["value"],
                                  "hansard": extract_optional(mp, "hansard")} for mp in wikidata_data["results"]["bindings"] ])

    # Format date of birth/death columns as datetime
    wikidata_mps.dob = pd.to_datetime(wikidata_mps.dob.fillna("").apply(lambda x: x[:-len("T00:00:00Z")]), errors="coerce")
    wikidata_mps.dod = pd.to_datetime(wikidata_mps.dod.fillna("").apply(lambda x: x[:-len("T00:00:00Z")]), errors="coerce")

    # Filter out any MPs that were dead by 1970 (they couldn't possibly appear in parliament speeches post 1970)
    wikidata_mps = wikidata_mps[(wikidata_mps.dod > "1969") | wikidata_mps.dod.isnull()]

#### Some MPs on wikidata don't have a DOB or DOD so get them from Hansard using the identifier

In [8]:
def get_dob_from_hansard(mp_id):
    """Go to MP's hansard page and scrape DOB, DOD and constituencies served"""
    import requests
    from bs4 import BeautifulSoup
    
    hansard_request = requests.get("http://hansard.millbanksystems.com/people/" + mp_id)
    soup = BeautifulSoup(hansard_request.content, 'html.parser')
    
    # Find DOB and DOD by looking for vcard tag and taking the next line
    try:
        dates = soup.select("h1.vcard")[0].next_sibling.strip()
    except IndexError:
        dates = ""
    # Find constituencies served by MP
    # Return both in a dict
    return pd.Series({"dates":dates,
            "constituencies": [constituency.text for constituency in soup.select("li.constituency a")]})


if True:
    # Do it with multiple processes
    from multiprocessing import Pool
    pool = Pool(16)

    # Select MPs that do not have DOB or DOD but do have a hansard id
    mps_to_scrape = wikidata_mps[wikidata_mps.hansard.notnull()]
    # Run scrape function on all of them
    mps_scraped = pd.DataFrame(pool.map(get_dob_from_hansard, list(mps_to_scrape.hansard)))
    pool.close()
    pool.join()
    # Copy the index over so that the rows match
    mps_scraped.index = mps_to_scrape.index
    # Split the date text into DOB and DOD and assign to new columns
    mps_hansard = pd.concat([mps_to_scrape, mps_scraped\
                         .apply(lambda x: pd.Series(sum([str(x["dates"]).split("-"),
                                                         [x["constituencies"]]], [])), axis=1)\
                         .rename(columns={0:"hansard_dob", 1:"hansard_dod", 2:"hansard_constituencies"})],
                            axis=1)
    # Format dates as datetime
    mps_hansard["hansard_dob"] = pd.to_datetime(mps_hansard["hansard_dob"], errors="coerce") # If only year is recorded, it uses the 1st of January
    mps_hansard["hansard_dod"] = pd.to_datetime(mps_hansard["hansard_dod"], errors="coerce") # If only year is recorded, it uses the 1st of January

#### Bring this hansard data back into the wikidata data frame and add a new column using mp name and year of birth to get a unique identifier for mp

In [9]:
if True:
    # Bring hansard data back into wikidata dataframe
    wikidata_mps = wikidata_mps.join(mps_hansard[["hansard_dob", "hansard_dod", "hansard_constituencies"]])

    # Fill in empty DOB and DOD using hansard data
    wikidata_mps["dob"] = wikidata_mps["dob"].fillna(wikidata_mps["hansard_dob"])
    wikidata_mps["dod"] = wikidata_mps["dod"].fillna(wikidata_mps["hansard_dod"])

    # Create a year of birth column (can't use full date of birth because older MPs don't have accurate DOB or only have year of birth)
    wikidata_mps["yob"] = pd.to_numeric(wikidata_mps["dob"].dt.year.fillna(0).astype("int"))

    # Use both mp name and year of birth to group so that we don't accidentally match different MPs
    wikidata_mps["mp_yob"] = wikidata_mps["mp"] + "_" +wikidata_mps["yob"].astype(str)

#### Wikidata gives us multiple rows per MP so collapse into one row without throwing away any useful info

In [10]:
def flatten_mps(mp_group):
    """Flatten a group of rows belonging to one MP into one row"""
    flattened_data = pd.Series()
    try:
        flattened_data["dob"] = pd.to_datetime(mp_group["dob"].dropna().iloc[0])
    except IndexError:
        flattened_data["dob"] = pd.NaT
    try:
        flattened_data["dod"] = pd.to_datetime(mp_group["dod"].dropna().iloc[0])
    except IndexError:
        flattened_data["dod"] = pd.NaT
    flattened_data["gender"] = mp_group["gender"].iloc[0]
    flattened_data["mp"] = mp_group["mp"].iloc[0]
    flattened_data["party"] = mp_group["party"].unique().tolist()
    flattened_data["mp_id"] = mp_group["mp_id"].unique().tolist()[0]
    try:
        # Collapse all lists of constituencies into one set of constituencies
        flattened_data["hansard_constituencies"] = set(sum(mp_group["hansard_constituencies"].dropna().tolist(), []))
    except TypeError:
        print(mp_group)
    
    return flattened_data

if True:
    # Filter by popular parties only
    wikidata_mps = wikidata_mps[wikidata_mps["party"].isin(
        ['Labour Party',
     'Conservative Party',
     'Plaid Cymru',
     'Liberal Democrats',
     'Liberal Party',
     'Ulster Unionist Party',
     'Scottish National Party',
     'Labour Co-operative',
     'Democratic Unionist Party',
     'Social Democratic Party',
     'Social Democratic and Labour Party',
     'UK Independence Party',
     'Green Party',
     'Sinn Féin',
     'Alliance Party of Northern Ireland',
     'Respect Party',
     'Co-operative Party'])]\
    .groupby("mp_yob").apply(flatten_mps) # Collapse all the duplicate rows into one per MP.
                                          # Constituencies are returned as sets, parties are returned as lists
    
    # Change them to datetime again...
    wikidata_mps["dob"] = pd.to_datetime(wikidata_mps["dob"])
    wikidata_mps["dod"] = pd.to_datetime(wikidata_mps["dod"])

#### Manually correct some data so that joining goes more smoothly

In [11]:
if True:
    # Add some MP constituencies manually so that the matching goes smoothly
    wikidata_mps.query("mp_yob=='Angela Smith_1961'")["hansard_constituencies"].iloc[0].add("Penistone and Stocksbridge")
    wikidata_mps.query("mp=='John Foster'")["hansard_constituencies"].iloc[0].add("Northwich")
    wikidata_mps.query("mp=='Alan Brown'")["hansard_constituencies"].iloc[0].add("Kilmarnock and Loudoun")
    wikidata_mps.query("mp_yob=='Mike Wood_1976'")["hansard_constituencies"].iloc[0].add("Dudley South")
    wikidata_mps.query("mp=='Neil Carmichael'")["hansard_constituencies"].iloc[0].add("Stroud")
    wikidata_mps.query("mp=='Iain Stewart'")["hansard_constituencies"].iloc[0].add("Milton Keynes South")
    wikidata_mps.query("mp=='Donald Stewart'")["hansard_constituencies"].iloc[0].add("Na h-Eileanan an Iar")
    wikidata_mps.query("mp_yob=='Stewart McDonald_1986'")["hansard_constituencies"].iloc[0].add("Glasgow South")
    wikidata_mps.query("mp_yob=='Ian Paisley, Jr._1966'")["hansard_constituencies"].iloc[0].add("North Antrim")
    wikidata_mps.query("mp_yob=='Geoffrey Clifton-Brown_1953'")["hansard_constituencies"].iloc[0].add("The Cotswolds")
    mps.loc[mps["full_name"]=='Ian Paisley Jnr', "full_name"] = "Ian Paisley, Jr."

#### Now match wikidata dataframe with the original list of MPs.

##### Function to match an MP and disambiguate between similar MPs

In [12]:
def match_mp(mp):
    """Use fuzzy string matching to match an MP by name.
    If there are multiple MPs with similar names, try to disambiguate using DOB and constituencies"""

    from fuzzywuzzy import fuzz, process
    
    matched_mps = process.extractBests(mp["full_name"], wikidata_mps["mp"],
                             scorer=fuzz.partial_token_sort_ratio, # computationally expensive scorer, but works well
                             score_cutoff=95)
    matched_mps_2 = process.extractBests(mp["full_name"], wikidata_mps["mp"],
                             scorer=fuzz.token_set_ratio, # alternate scorer which also works well
                             score_cutoff=95)
    # Combine both types of matches to get best results
    matched_mps.extend(matched_mps_2)
    # Extract mp_yob key and put in set
    matched_mps = {mp[2] for mp in matched_mps}
    # If there is only one fuzzy match, then assume we have our MP

    if len(matched_mps) == 1:
        # Get party from wikidata table
        party = wikidata_mps["party"].loc[list(matched_mps)[0]]
        # Get wikidata id from table
        mp_id = wikidata_mps["mp_id"].loc[list(matched_mps)[0]]
        return (party, mp_id, list(matched_mps)[0])
    elif len(matched_mps) > 1:
        # We have several matches so let's try to disambiguate by constituencies served
        # From the wikidata table, find these matched MP_yods, 
        matched_mp_yobs =  wikidata_mps.loc[matched_mps]
        # Then check if requested mp's constituency is in the list of constituencies of each match
        matched_mps_in_wiki = matched_mp_yobs["hansard_constituencies"]\
            .apply(lambda x: (process.extractOne(mp["Constituency"], x, scorer=fuzz.token_set_ratio)  or [(), (-1)])[1] > 90)
        # Now filter by Trues and get a good match
        try:
            matched_mps = matched_mps_in_wiki.where(lambda x: x==True).dropna().index
            # if there are multiple matches, match by exact surname
            for match in matched_mps:
                if ", Jr." in match:
                    if match.split("_")[0] == mp["full_name"]:
                        matched_mp = match
                        break
                elif match.split("_")[0].split(",")[0].split(" ")[-1] == mp["full_name"].split(",")[0].split(" ")[-1]:
                    matched_mp = match
                    break
            party = wikidata_mps["party"].loc[matched_mp]
            mp_id = wikidata_mps["mp_id"].loc[matched_mp]
        except UnboundLocalError:
            print(mp, matched_mps)
            raise UnboundLocalError
        return (party, mp_id, matched_mp)
    else:
        return (None, None, None)

##### Now run the above matching function for all MPs

In [13]:
if True:
    from multiprocessing import Pool

    # Create a pool of 8 processes
    pool = Pool(8)
    
    # Find 100% match, if it exists and add it to the MPs dataframe
    mps_wikidata = list(pool.map(match_mp, mps[["full_name", "Constituency"]].to_dict("records")))
    pool.close()
    pool.join()
    # Messy step to clean up the matches...
    mps_wikidata = list(zip(*[i if i != None else (None, None, None) for i in mps_wikidata]))
    mps = mps.assign(party_wikidata=mps_wikidata[0], mp_wikidata_id=mps_wikidata[1], mp_wikidata=mps_wikidata[2])

    # Map party names in wikidata to standardised party names
    party_abbr_wiki = {
    'Labour Party':"Labour",
     'Conservative Party':"Conservative",
     'Plaid Cymru':"Plaid Cymru",
     'Liberal Democrats':"Liberal Democrat",
     'Liberal Party':"Liberal Party",
     'Ulster Unionist Party':"UUP",
     'Scottish National Party':'Scottish National Party',
     'Democratic Unionist Party': "DUP",
     'Social Democratic Party': "SDP",
     'Social Democratic and Labour Party': "Social Democratic and Labour Party",
     'UK Independence Party': "UKIP",
     'Green Party': "Green",
     'Sinn Féin': "Sinn Féin",
     'Alliance Party of Northern Ireland': "Alliance",
     'Respect Party': "Respect",
     'Labour/Co-operative': "Labour/Co-operative",
     'Co-operative Party': "Labour/Co-operative"
    }
    # For MPs that do not have a party affiliation, use the party affiliation from mysociety
    mps.Party = mps.Party.fillna(mps["party_mysoc"])

#### For the remaining MPs without several party affiliations, look up the MP on wikipedia, and select the most relevant party as a last resort

In [14]:
# For the MPs that have wikidata info but no party, find their party
def scrape_mp_current_party(mp_id):
    from bs4 import BeautifulSoup
    """Look up MP's wikipedia page, then go down the table of data for MP and try to figure out current party.
    mp_id: the mp _ year of death key used in wikidata_mps data frame"""
    mp_wikidata_id = wikidata_mps.loc[mp_id]["mp_id"].split("/")[-1]
    parties = wikidata_mps.loc[mp_id]["party"]
    if len(parties) < 2:
        return parties
    wikipedia_name = requests.get("https://www.wikidata.org/w/api.php?action=wbgetentities&format=xml&props=sitelinks&ids={0}&sitefilter=enwiki".format(mp_wikidata_id))
    wikipedia_src = requests.get("https://en.wikipedia.org/wiki/" + str(BeautifulSoup(wikipedia_name.text, "html5lib").select("sitelink")[0]["title"]))
    print(wikipedia_src.url)
    for row in BeautifulSoup(wikipedia_src.text, "html5lib").select("table.infobox tr"):
        for party in parties:
            # Return first party that is found.
            # Assume that the party near the top of wikipedia data table is the MP's current party
            if (party in row.text) | (party.replace("Democrats", "Democrat") in row.text):
                return [party]
    return parties

if True:
    # Filter only the MPs that need wikidata info
    mps_filter = mps.party_wikidata.notnull() & mps.Party.isnull()
    mps.loc[mps_filter, "party_wikidata"] = mps.loc[mps_filter, "mp_wikidata"].apply(scrape_mp_current_party)

    # Assume that all Liberal MPs eventually became Liberal Democrats
    mps.loc[mps_filter, "party_wikidata"] = mps.loc[mps_filter, "party_wikidata"].apply(lambda x: ["Liberal Democrats"] if x == ["Liberal Democrats", "Liberal Party"] else x)

https://en.wikipedia.org/wiki/Jim_Sillars
https://en.wikipedia.org/wiki/Neil_Hamilton_(politician)
https://en.wikipedia.org/wiki/Roy_Jenkins
https://en.wikipedia.org/wiki/David_Alton
https://en.wikipedia.org/wiki/Shirley_Williams
https://en.wikipedia.org/wiki/Alan_Amos
https://en.wikipedia.org/wiki/Jack_Aspinwall
https://en.wikipedia.org/wiki/Rhodes_Boyson
https://en.wikipedia.org/wiki/Michael_Brotherton
https://en.wikipedia.org/wiki/Edmund_Dell
https://en.wikipedia.org/wiki/Charles_Fletcher-Cooke
https://en.wikipedia.org/wiki/Jo_Grimond
https://en.wikipedia.org/wiki/Emlyn_Hooson,_Baron_Hooson
https://en.wikipedia.org/wiki/Roger_Knapman
https://en.wikipedia.org/wiki/Dickson_Mabon
https://en.wikipedia.org/wiki/David_Marquand
https://en.wikipedia.org/wiki/Christopher_Mayhew
https://en.wikipedia.org/wiki/Anna_McCurley
https://en.wikipedia.org/wiki/Tom_McNally,_Baron_McNally
https://en.wikipedia.org/wiki/Sir_Anthony_Meyer,_3rd_Baronet
https://en.wikipedia.org/wiki/Michael_O%27Halloran_(Bri

#### Now that we have affiliations from different sources, let's use them to fill in the gaps in this order:
1. Wikidata
2. Women MPs

Finally, we're done! Let's save it to a HDF5 list

In [18]:
if True:
    # Flatten remaining party lists and for lists with several parties, just take the first one
    mps["party_wikidata"] = mps["party_wikidata"].apply(lambda x: None if x == None else party_abbr_wiki.get(x[0]))
    # If an MP doesn't have a party defined, then use one of the other sources to assign a party
    mps.Party = mps.Party.fillna(mps["party_wikidata"]).fillna(mps["party_women"])

    # Correct Sinn Féin spelling
    mps.loc[mps.Party == "Sinn Fein", "Party"] = "Sinn Féin"
    
    # Clean up some MP IDs that are different to the ones in speeches
    mps = mps.set_index("Person ID").reset_index()
    mps.loc[mps.query("mp_wikidata == 'Winnie Ewing_1929'").index[0], "Person ID"] = 22574
    mps.loc[mps.query("mp_wikidata == 'James Callaghan_1912'").index[0], "Person ID"] = 16877
    mps = mps.set_index("Person ID")
    
    # Save MPs to disk
    mps.to_hdf("list_of_mps.h5", "mps", mode="w")
else:
    mps = pd.read_hdf("list_of_mps.h5", "mps")

/home/durand/Stuff/Sources/anaconda3/envs/nlp/lib/python3.6/site-packages/pandas/core/generic.py:1299: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['First name', 'Last name', 'Party', 'Constituency', 'URI', 'full_name', 'clean_name', 'party_women', 'party_mysoc', 'mp_wikidata', 'mp_wikidata_id', 'party_wikidata']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [19]:
mps

,First name,Last name,Party,Constituency,URI,full_name,clean_name,is_female,party_women,party_mysoc,mp_wikidata,mp_wikidata_id,party_wikidata
Person ID,,,,,,,,,,,,,
10001,Diane,Abbott,Labour,Hackney North and Stoke Newington,https://www.theyworkforyou.com/mp/10001/diane_...,Diane Abbott,dianeabbott,True,Labour,Labour,Diane Abbott_1953,http://www.wikidata.org/entity/Q153454,Labour
10002,Gerry,Adams,Sinn Féin,Belfast West,https://www.theyworkforyou.com/mp/10002/gerry_...,Gerry Adams,gerryadams,False,None,Sinn Féin,Gerry Adams_1948,http://www.wikidata.org/entity/Q76139,Sinn Féin
10003,Irene,Adams,Labour,Paisley North,https://www.theyworkforyou.com/mp/10003/mrs_ir...,Irene Adams,ireneadams,True,Labour,Labour,"Irene Adams, Baroness Adams of Craigielea_1947",http://www.wikidata.org/entity/Q334498,Labour
10004,Nick,Ainger,Labour,Carmarthen West and South Pembrokeshire,https://www.theyworkforyou.com/mp/10004/nick_a...,Nick Ainger,nickainger,False,None,Labour,Nick Ainger_1949,http://www.wikidata.org/entity/Q325367,Labour
10005,Bob,Ainsworth,Labour,Coventry North East,https://www.theyworkforyou.com/mp/10005/bob_ai...,Bob Ainsworth,bobainsworth,False,None,Labour,Bob Ainsworth_1952,http://www.wikidata.org/entity/Q258738,Labour
10006,Peter,Ainsworth,Conservative,East Surrey,https://www.theyworkforyou.com/mp/10006/peter_...,Peter Ainsworth,peterainsworth,False,None,Conservative,Peter Ainsworth_1956,http://www.wikidata.org/entity/Q337709,Conservative
10007,Richard,Allan,Liberal Democrat,"Sheffield, Hallam",https://www.theyworkforyou.com/mp/10007/mr_ric...,Richard Allan,richardallan,False,None,Liberal Democrat,"Richard Allan, Baron Allan of Hallam_1966",http://www.wikidata.org/entity/Q334455,Liberal Democrat
10008,Graham,Allen,Labour,Nottingham North,https://www.theyworkforyou.com/mp/10008/graham...,Graham Allen,grahamallen,False,None,Labour,Graham Allen_1953,http://www.wikidata.org/entity/Q259601,Labour
10009,David,Amess,Conservative,Southend West,https://www.theyworkforyou.com/mp/10009/david_...,David Amess,davidamess,False,None,Conservative,David Amess_1952,http://www.wikidata.org/entity/Q259646,Conservative
